# SGNet Trainer

In [1]:
import matplotlib.pyplot as pp

In [2]:
from train import *
from sgnet import *
from generator.generator import *

In [3]:
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(asctime)s %(message)s', level=logging.INFO) 

In [11]:
import torch
x = torch.tensor([
    [0],
    [1],
    [2],
    [3],
    [4],
])
print(x.shape)

torch.Size([5, 1])


In [13]:
k = x.repeat(1, 4)
k.shape

torch.Size([5, 4])

# Setup model configs

In [4]:
def get_mock_config():
    return {
        'base_lr': 0.0005,
        'num_data_loaders' : 4,     
        'num_training_epochs' : 20,
        'batch_size' : 32,
        'pin_memory': True,
        'use_apex' : False,
        'warmup_period_for_lr' : 1000000, 
        'decay_interval' : 0,
        'decay' : 0.8,
        'l2_reg' : 1e-4,
        'resource_path': 'J:\\datasets\\faze-resources',
        'npz_root_path': 'J:\\datasets\\everyone-recode',
    }

# Build context

In [5]:
config = get_mock_config()
Context.build(config)
ctx = Context.get()

# Build network

In [6]:
network_key = "ss"
INPUT_SIZE = (3, 64, 64)
network = SGNet(INPUT_SIZE, network_key)

In [7]:
ctx.set_network(network_key, network)
ctx.load_network_to_device()

# Setup Base Loss Policy

In [8]:
ctx.setup_base_loss_policy(verbose=True)

2020-01-26 19:27:27,693 
2020-01-26 19:27:27,696 >>> base lose policy <<<
2020-01-26 19:27:27,697 max learning rate: 0.016000
2020-01-26 19:27:27,698 ramp up a: 0.000000, b: 0.000500


# Build Optimizer

In [9]:
optimizer = ctx.build_optimizer(network_key)
ctx.set_main_optimizer(optimizer)

2020-01-26 19:27:27,704 Set optimizer as key: optimizer


# Setup Loss function

In [10]:
loss_functions = ctx.make_loss_functions()
ctx.set_loss_functions(loss_functions)

2020-01-26 19:27:27,711 
2020-01-26 19:27:27,712 >>> Setup loss functions <<<
2020-01-26 19:27:27,713 list: { gaze } 


# Setup data-generator

In [13]:
def selector(self, idx):
    profile_id, item_idx = self.index_to_query[idx]    
    profile_idx = self.profiles.index("{:05d}".format(profile_id))
    npz = self.load_recode_profile_npz(self.npz_root_path, profile_id)

# gen = NPZDatasetGenerator(ctx, shuffle_train=False, item_selector_fn=selector)
gen = NPZDatasetGenerator(ctx, shuffle_train=False)
databag = gen.generate(verbose=False)
ctx.set_databag(databag)

train_ds, loader = ctx.get_dataset_and_loader(Context.TAG_TRAIN)
# train_ds.index_to_query[1900]

2020-01-26 19:27:37,465 
2020-01-26 19:27:37,466 >>> data-bag is setup
2020-01-26 19:27:37,467 => { gc/train, gc/val, gc/test } 


# Train

In [ ]:
# ctx.use_multiple_gpu_if_available()
# ctx.run_train(network_key)

In [ ]:
# execute_training_step(current_step)

# if current_step % args.print_freq_train == args.print_freq_train - 1:
#     conv1_wt_lr = optimizer.param_groups[0]['lr']
#     running_loss_means = running_losses.means()
#     logging.info('Losses at [%7d]: %s' %
#                  (current_step + 1,
#                   ', '.join(['%s: %.5f' % v
#                              for v in running_loss_means.items()])))
#     if args.use_tensorboard:
#         tensorboard.add_scalar('train_lr', conv1_wt_lr, current_step + 1)
#         for k, v in running_loss_means.items():
#             tensorboard.add_scalar('train/' + k, v, current_step + 1)
#     running_losses.reset()

# # Print some timing statistics
# if current_step % 100 == 99:
#     if args.use_tensorboard:
#         for k, v in running_timings.means().items():
#             tensorboard.add_scalar('timing/' + k, v, current_step + 1)
#     running_timings.reset()

# # print some memory statistics
# if current_step % 5000 == 0:
#     for i in range(torch.cuda.device_count()):
#         bytes = (torch.cuda.memory_allocated(device=i)
#                  + torch.cuda.memory_cached(device=i))
#         logging.info('GPU %d: probably allocated approximately %.2f GB' % (i, bytes / 1e9))
